In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Q1= reading files
akas = pd.read_csv('title.akas.tsv', delimiter = "\t", dtype = 'object')
akas.head(10)
#basics = pd.read_csv('title.basics.tsv', delimiter = '\t','object')
#ratings = pd.read_csv('title.ratings.tsv', delimiter = '\t','object')

In [ ]:
#Q2 =Dropping duplicates
akas.drop_duplicates()
basics.drop_duplicates()
ratings.drop_duplicates()

In [ ]:
#Q3 = Using merge
#Q3.1 Inner merge
newdf = pd.merge(basics, ratings, on ='tconst', how ='inner')
movies_mergedInner = pd.merge(akas, newdf, left_on='titleId', right_on='tconst', how ='inner')
#number of lines 1686547, 19 columns
#Q3.1 Outer merge
df = pd.merge(basics, ratings, on ='tconst', how ='outer')
movies_mergedOuter = pd.merge(akas, df, left_on='titleId', right_on='tconst', how ='outer')
# number of lines 3419932, 19 columns
#Q3.2 Unique method
unique= movies_mergedInner['titleType'].nunique()
print(unique)

In [ ]:
#Q4
df_new = movies_mergedOuter[(movies_mergedOuter.titleType=='movie') & 
                            ((movies_mergedOuter.region=='US') |
                             (movies_mergedOuter.language =='en'))]

In [ ]:
#Q5 add log10votes to datatframe
df_new['log10Votes'] = np.log10(df_new['numVotes'])

In [ ]:
#Q6 Lower case of genres column
df_new.genres = df_new.genres.str.lower()

In [ ]:
#Q7 aggregate by genre and show highest 10
#Q71 by mean of log10VOtes
df1 = df_new.groupby('genres')['log10Votes'].mean().sort_values(ascending=False)
df1.head(10)
#Q7 by mean of averageRating
df2 = df_new.groupby('genres')['averageRating'].mean().sort_values(ascending=False)
df2.head(10)